In [1]:
# --- Standard Library ---
from pathlib import Path

# --- Third-Party Libraries ---
import pandas as pd
import sidetable as stb

In [2]:
# Set the maximum number of columns to display 
pd.set_option('display.max_columns', None)

## Leitura inicial e concatenação dos dados

In [3]:
all_data_dict = {}

In [4]:
BASE_DIR = Path("../data/raw")  

# Dictionary of files
csv_files = {
    'srag_19': "INFLUD19-26-06-2025.csv",
    'srag_20': "INFLUD20-26-06-2025.csv",
    'srag_21': "INFLUD21-26-06-2025.csv",
    'srag_22': "INFLUD22-26-06-2025.csv",
    'srag_23': "INFLUD23-26-06-2025.csv",
    'srag_24': "INFLUD24-26-06-2025.csv",
    'srag_25': "INFLUD25-22-09-2025.csv",
}

all_data_dict = {}

for key, filename in csv_files.items():
    file_path = BASE_DIR / filename  
    all_data_dict[key] = pd.read_csv(file_path, sep=';', low_memory=False)

# Test
print(all_data_dict['srag_19'].head())

     NU_NOTIFIC  DT_NOTIFIC  SEM_NOT  DT_SIN_PRI  SEM_PRI SG_UF_NOT  \
0  315478195042  2019-01-10        2  2019-01-06        2        MG   
1  315478195276  2019-01-03        1  2019-01-01        1        SP   
2  315478207219  2019-01-02        1  2018-12-31        1        PE   
3  315478211086  2019-01-10        2  2019-01-07        2        SP   
4  315478212765  2019-01-11        2  2019-01-06        2        PE   

          ID_REGIONA  CO_REGIONA      ID_MUNICIP  CO_MUN_NOT CS_SEXO  \
0     BELO HORIZONTE      1449.0  BELO HORIZONTE    310620.0       M   
1      GVE I CAPITAL      1331.0       SAO PAULO    355030.0       F   
2                001      1497.0          RECIFE    261160.0       M   
3  GVE XVII CAMPINAS      1342.0        CAMPINAS    350950.0       F   
4                004      1499.0         CARUARU    260410.0       F   

      DT_NASC  NU_IDADE_N  TP_IDADE  COD_IDADE  CS_GESTANT  CS_RACA CS_ETINIA  \
0  1988-03-17          30         3       3030           6 

Checando se existem colunas divergentes entre uma base e outra

In [5]:
# Pegamos as colunas do primeiro DataFrame como referência
cols_ref = list(all_data_dict[list(all_data_dict.keys())[0]].columns)

# Checamos se todos os DataFrames têm as mesmas colunas
if all(list(df.columns) == cols_ref for df in all_data_dict.values()):
    print("✅ All DataFrames have the same columns")
else:
    print("⚠️ Some DataFrames have different columns")

✅ All DataFrames have the same columns


## 📊 Seleção de Colunas e Justificativa

O conjunto de dados possui **194 colunas**. Após análise, selecionamos apenas as colunas essenciais para as métricas solicitadas, garantindo que cada cálculo seja baseado em informações confiáveis. As variáveis selecionadas precisam contemplar a necessidade de calcular cada um dos indicadores:

#### 🔹 Taxa de Aumento de Casos
#### 🔹 Taxa de Mortalidade
#### 🔹 Taxa de Ocupação de UTI
#### 🔹 Taxa de Vacinação

#### Para os indicadores, foram selecionadas as colunas:

- `NM_PACIENT` → nome do paciente, também pode ser útil para verificar duplicatas 
- `DT_NOTIFIC` → data da notificação do caso (preenchimento da ficha de notificação)
- `VG_ENC` → encerramento do caso (se confirmado por sequenciamento genômico, sugestivo ou 5 - descartado)
- `CLASSI_FIN` → diagnóstico final do caso (tipo de SRAG diagnosticada)  
- `EVOLUCAO` → evolução do caso 
    - 1 - cura | 2 - óbito | 3 - óbito por outras causas | 9-ignorado  
- `UTI` → se o paciente foi internado em UTI 
    - 1 - sim | 2 - não | 9 - ignorado  
- `VACINA` → se o paciente era vacinado ou não contra gripe (checagem de caderneta ou informação fornecida pelo paciente)
    - 1 - sim | 2 - não | 9 - ignorado  
- `VACINA_COV` → se o paciente recebeu vacina ou não (COVID - checagem da caderneta)
    - 1 - sim | 2 - não | 9 - ignorado  

#### Também, foram selecionadas outras colunas que podem ser úteis (para filtros e estatísticas):

- `DT_NASC` → data de nascimento
- `NU_IDADE_N` → idade numérica  
- `CS_SEXO` → sexo  
- `CS_ZONA` → zona urbana/rural 
- `CO_MUN_RES` → município de residência (opcional para agregação por cidade)  
- `CO_MUN_NOT` → município de notificação  
- `SG_UF` → estado  

**Justificativa:** informações demográficas e geográficas ajudam em agregações e análises complementares.


Selecionando apenas as colunas desejadas

In [6]:
# Lista de todas as colunas que serão mantidas
cols_to_keep = [
    "NU_NOTIFIC", "DT_NOTIFIC", "VG_ENC" ,"CLASSI_FIN", "EVOLUCAO", "DT_ENCERRA",
    "UTI", "VACINA", "VACINA_COV", "DT_NASC", "NU_IDADE_N",
    "CS_SEXO", "CS_ZONA", "CO_MUN_RES", "CO_MUN_NOT", "SG_UF",
]

# Filtrando todos os DataFrames do dicionário
all_data_dict_filtered = {year: df[cols_to_keep].copy() for year, df in all_data_dict.items()}

Gerando um dataframe único com a informação de todos os anos

In [7]:
df_srag = pd.concat(
    all_data_dict_filtered.values(),
    keys=all_data_dict_filtered.keys(),
    names=["source", None]
).reset_index(level="source").reset_index(drop=True)


Checando se a operação foi bem sucedida

In [8]:
for df, values in all_data_dict_filtered.items():

    print(f"{df} : {len(values)} values")

srag_19 : 48961 values
srag_20 : 1206920 values
srag_21 : 1745672 values
srag_22 : 560577 values
srag_23 : 279453 values
srag_24 : 267984 values
srag_25 : 256401 values


In [9]:
df_srag["source"].value_counts(dropna=False).reset_index().sort_values(by="source")

,source,count
6,srag_19,48961
1,srag_20,1206920
0,srag_21,1745672
2,srag_22,560577
3,srag_23,279453
4,srag_24,267984
5,srag_25,256401


## Análise qualitativa dos dados

In [10]:
df_srag.stb.missing()

,missing,total,percent
VG_ENC,4358216,4365968,99.822445
VACINA_COV,1531260,4365968,35.072635
VACINA,1181001,4365968,27.050152
UTI,574681,4365968,13.162740
CS_ZONA,444682,4365968,10.185187
EVOLUCAO,327471,4365968,7.500536
DT_ENCERRA,315672,4365968,7.230287
CLASSI_FIN,169612,4365968,3.884866
CO_MUN_NOT,10581,4365968,0.242352
DT_NASC,3880,4365968,0.088869


VG_ENC com muitos nulos, avaliando a distribuição:

In [11]:
df_srag["VG_ENC"].value_counts(dropna=False)

VG_ENC
NaN    4358216
3.0       4796
1.0       1896
5.0        874
4.0        127
2.0         59
Name: count, dtype: int64

Casos com o valor 5 (descartado) em tese podem ser eliminados da base. Depois disso, a coluna não tem mais utilidade. 

In [12]:
df_srag = df_srag[df_srag["VG_ENC"] != 5].copy()
df_srag = df_srag.drop(columns="VG_ENC")

Avaliando a necessidade de utilizar ambas as colunas VACINA e VACINA_COV 

In [13]:
df_srag["VACINA"].value_counts(dropna=False)

VACINA
2.0    1405254
9.0    1304791
NaN    1180834
1.0     474215
Name: count, dtype: int64

In [14]:
df_srag["VACINA_COV"].value_counts(dropna=False)

VACINA_COV
NaN    1531260
2.0    1236045
1.0    1230921
9.0     366868
Name: count, dtype: int64

In [15]:
df_srag[df_srag["VACINA_COV"] == 1]["VACINA"].value_counts(dropna=False)

VACINA
2.0    418007
NaN    312802
9.0    311443
1.0    188669
Name: count, dtype: int64

Pelo comportamento dos dados, são informações complementares.

Em um primeiro momento, entende-se que os valores nulos podem ser interpretados com o rótulo 9 - ignorado/sem informação

O mesmo se aplica para as colunas `UTI`| `CS_ZONA`| `EVOLUCAO`| 


In [16]:
df_srag["VACINA"] = df_srag["VACINA"].fillna(9)
df_srag["VACINA_COV"] = df_srag["VACINA_COV"].fillna(9)
df_srag["UTI"] = df_srag["UTI"].fillna(9)
df_srag["CS_ZONA"] = df_srag["CS_ZONA"].fillna(9)
df_srag["EVOLUCAO"] = df_srag["EVOLUCAO"].fillna(9)

Checando eventuais notificações duplicadas 

In [17]:
abs(df_srag['NU_NOTIFIC'].nunique() - len(df_srag))

11

Existem notificações duplicadas, mas casos bem pontuais (9). Verificando os casos

In [18]:
df_srag[df_srag["NU_NOTIFIC"].duplicated(keep=False)].sort_values(by="NU_NOTIFIC")

,source,NU_NOTIFIC,DT_NOTIFIC,CLASSI_FIN,EVOLUCAO,DT_ENCERRA,UTI,VACINA,VACINA_COV,DT_NASC,NU_IDADE_N,CS_SEXO,CS_ZONA,CO_MUN_RES,CO_MUN_NOT,SG_UF
4101908,srag_24,31735935004234,2024-01-03,NaN,9.0,NaN,9.0,9.0,2.0,2007-12-23,16,F,1.0,230440.0,230440.0,CE
4110055,srag_25,31735935004234,2025-01-03,4.0,1.0,2025-07-25,9.0,9.0,2.0,2007-12-23,16,F,1.0,230440.0,230440.0,CE
4104124,srag_24,31736722248953,2025-01-12,4.0,2.0,2025-01-12,9.0,1.0,1.0,1973-01-14,50,M,1.0,353440.0,353440.0,SP
4112461,srag_25,31736722248953,2025-01-12,4.0,3.0,2025-01-12,9.0,1.0,1.0,1973-01-14,51,M,1.0,353440.0,353440.0,SP
4106818,srag_24,31738932515531,2024-02-07,4.0,9.0,NaN,2.0,9.0,2.0,2018-11-19,5,M,1.0,530040.0,530010.0,DF
4123097,srag_25,31738932515531,2025-02-07,4.0,1.0,2025-08-18,2.0,9.0,2.0,2018-11-19,5,M,1.0,530040.0,530010.0,DF
4109076,srag_24,31743770891762,2024-04-04,NaN,9.0,NaN,9.0,2.0,2.0,1987-02-02,37,M,1.0,351060.0,353440.0,SP
4155998,srag_25,31743770891762,2025-04-04,4.0,2.0,2025-05-04,9.0,2.0,2.0,1987-02-02,37,M,1.0,351060.0,353440.0,SP
4109180,srag_24,31744397861067,2025-04-11,4.0,1.0,2025-06-03,1.0,2.0,1.0,1968-02-21,56,M,1.0,352400.0,352590.0,SP
4162344,srag_25,31744397861067,2025-04-11,4.0,1.0,2025-06-03,1.0,2.0,1.0,1968-02-21,56,M,1.0,352400.0,352590.0,SP


Eliminando duplicatas gerais (informações/ linhas inteiras duplicadas)

In [19]:
len(df_srag)

4365094

In [20]:
columns_to_consider = [col for col in df_srag.columns if "source" not in col]

df_srag = df_srag.drop_duplicates(subset=columns_to_consider)

In [21]:
len(df_srag)

4365090

In [22]:
df_srag[df_srag["NU_NOTIFIC"].duplicated(keep=False)].sort_values(by="NU_NOTIFIC")

,source,NU_NOTIFIC,DT_NOTIFIC,CLASSI_FIN,EVOLUCAO,DT_ENCERRA,UTI,VACINA,VACINA_COV,DT_NASC,NU_IDADE_N,CS_SEXO,CS_ZONA,CO_MUN_RES,CO_MUN_NOT,SG_UF
4101908,srag_24,31735935004234,2024-01-03,NaN,9.0,NaN,9.0,9.0,2.0,2007-12-23,16,F,1.0,230440.0,230440.0,CE
4110055,srag_25,31735935004234,2025-01-03,4.0,1.0,2025-07-25,9.0,9.0,2.0,2007-12-23,16,F,1.0,230440.0,230440.0,CE
4104124,srag_24,31736722248953,2025-01-12,4.0,2.0,2025-01-12,9.0,1.0,1.0,1973-01-14,50,M,1.0,353440.0,353440.0,SP
4112461,srag_25,31736722248953,2025-01-12,4.0,3.0,2025-01-12,9.0,1.0,1.0,1973-01-14,51,M,1.0,353440.0,353440.0,SP
4106818,srag_24,31738932515531,2024-02-07,4.0,9.0,NaN,2.0,9.0,2.0,2018-11-19,5,M,1.0,530040.0,530010.0,DF
4123097,srag_25,31738932515531,2025-02-07,4.0,1.0,2025-08-18,2.0,9.0,2.0,2018-11-19,5,M,1.0,530040.0,530010.0,DF
4109076,srag_24,31743770891762,2024-04-04,NaN,9.0,NaN,9.0,2.0,2.0,1987-02-02,37,M,1.0,351060.0,353440.0,SP
4155998,srag_25,31743770891762,2025-04-04,4.0,2.0,2025-05-04,9.0,2.0,2.0,1987-02-02,37,M,1.0,351060.0,353440.0,SP
4109220,srag_24,31744827529781,2025-04-14,NaN,9.0,NaN,1.0,1.0,1.0,2020-01-27,4,M,1.0,330455.0,330455.0,RJ
4166355,srag_25,31744827529781,2025-04-14,4.0,1.0,2025-04-16,1.0,2.0,1.0,2020-01-27,5,M,1.0,330455.0,330455.0,RJ


Eliminando duplicatas que possuem data de encerramento nulas (informação mais recente disponível na base)

In [23]:
df_srag = df_srag[
    ~(
        (df_srag["NU_NOTIFIC"].duplicated(keep=False))
        &
        (df_srag["DT_ENCERRA"].isna())
    )
].copy()

In [24]:
df_srag[df_srag["NU_NOTIFIC"].duplicated(keep=False)].sort_values(by="NU_NOTIFIC")

,source,NU_NOTIFIC,DT_NOTIFIC,CLASSI_FIN,EVOLUCAO,DT_ENCERRA,UTI,VACINA,VACINA_COV,DT_NASC,NU_IDADE_N,CS_SEXO,CS_ZONA,CO_MUN_RES,CO_MUN_NOT,SG_UF
4104124,srag_24,31736722248953,2025-01-12,4.0,2.0,2025-01-12,9.0,1.0,1.0,1973-01-14,50,M,1.0,353440.0,353440.0,SP
4112461,srag_25,31736722248953,2025-01-12,4.0,3.0,2025-01-12,9.0,1.0,1.0,1973-01-14,51,M,1.0,353440.0,353440.0,SP
4109291,srag_24,31745519106567,2025-04-24,4.0,1.0,2025-05-02,2.0,2.0,2.0,2023-09-12,7,F,9.0,330455.0,330455.0,RJ
4172812,srag_25,31745519106567,2025-04-24,4.0,1.0,2025-05-02,2.0,1.0,1.0,2023-09-12,7,F,1.0,330455.0,330455.0,RJ


Eliminando duplicatas por idade (considerar idade mais velha -> informação mais recente)

In [25]:
len(df_srag)

4365085

In [26]:
columns_to_consider = [col for col in df_srag.columns if "source" not in col and "NU_IDADE_N" not in col and "EVOLUCAO" not in col]

In [27]:
df_srag = df_srag.sort_values(by=["NU_IDADE_N"], ascending=False).drop_duplicates(subset=columns_to_consider)

In [28]:
len(df_srag)

4365084

In [29]:
df_srag[df_srag["NU_NOTIFIC"].duplicated(keep=False)].sort_values(by="NU_NOTIFIC")

,source,NU_NOTIFIC,DT_NOTIFIC,CLASSI_FIN,EVOLUCAO,DT_ENCERRA,UTI,VACINA,VACINA_COV,DT_NASC,NU_IDADE_N,CS_SEXO,CS_ZONA,CO_MUN_RES,CO_MUN_NOT,SG_UF
4109291,srag_24,31745519106567,2025-04-24,4.0,1.0,2025-05-02,2.0,2.0,2.0,2023-09-12,7,F,9.0,330455.0,330455.0,RJ
4172812,srag_25,31745519106567,2025-04-24,4.0,1.0,2025-05-02,2.0,1.0,1.0,2023-09-12,7,F,1.0,330455.0,330455.0,RJ


Eliminando duplicatas por vacinação ou não (caso todas as outras informações sejam iguais)

Nesse caso, considera-se a mais recente como positivo (vacinado)

In [30]:
columns_to_consider = [col for col in df_srag.columns if "source" not in col and "VACINA" not in col and "VACINA_COV" not in col and "CS_ZONA" not in col]

In [31]:
len(df_srag)

4365084

In [32]:
df_srag = df_srag.sort_values(by=["VACINA"]).drop_duplicates(subset=columns_to_consider)

df_srag = df_srag.sort_values(by=["VACINA_COV"]).drop_duplicates(subset=columns_to_consider)

In [33]:
len(df_srag)

4365083

In [34]:
df_srag["NU_NOTIFIC"].nunique() == len(df_srag)

True

Sem mais duplicatas. 

Exportando versão final para cálculo de métricas 

Relizando um filtro final para conter apenas dados de 2025 e 2024 (diminuir tamanho do arquivo)

In [35]:
df_srag["source"].value_counts(dropna=False)

source
srag_21    1745665
srag_20    1206918
srag_22     560558
srag_23     279067
srag_24     267682
srag_25     256232
srag_19      48961
Name: count, dtype: int64

In [36]:
df_srag = df_srag[df_srag["source"].isin(["srag_24", "srag_25"])].copy()

In [37]:
df_srag["source"].value_counts(dropna=False)

source
srag_24    267682
srag_25    256232
Name: count, dtype: int64

Convertendo a coluna de data para datetime e ordenando o dataframe com base nela

In [38]:
df_srag["DT_NOTIFIC"] = pd.to_datetime(df_srag["DT_NOTIFIC"], format="%Y-%m-%d")

df_srag = df_srag.sort_values(by="DT_NOTIFIC")

Eliminando a última data para evitar problemas (dados que podem ter sido inseridos no meio do dia, por exemplo)

In [40]:
last_date = df_srag["DT_NOTIFIC"].max()

df_srag = df_srag[df_srag["DT_NOTIFIC"] != last_date].copy()

In [42]:
EXP_DIR = Path("../data/processed")  

df_srag.to_parquet(f"{EXP_DIR}/df_srag.parquet")